In [1]:
import numpy as np
import pandas as pd

carData = pd.read_csv('motor_vehicles.csv')
print(carData.shape)  # gives a tuple with the shape of data frame

(1728, 7)


In [2]:
print(carData.head())

  fuel_efficiency maintenance doors    type   size safety RATING
0       excellent       vhigh   two  sports  small    low   JUNK
1       excellent       vhigh   two  sports  small    med   JUNK
2       excellent       vhigh   two  sports  small   high   JUNK
3       excellent       vhigh   two  sports    med    low   JUNK
4       excellent       vhigh   two  sports    med    med   JUNK


In [3]:
splitRatio = .85

In [4]:
fuel_efficienty_labels = ['excellent', 'good', 'med', 'crap']
maintenance_labels = ['low', 'med', 'high', 'vhigh']
doors_labels = ['two', 'three', 'four', 'many']
type_labels = ['sports', 'sedan', 'wagon']
size_labels = ['small', 'med', 'big']
safety_labels = ['low', 'med', 'high']

In [5]:
# Do the shuffle!
reCarData = carData.reindex(np.random.permutation(carData.index))
shuffCarData = reCarData.reset_index()

In [6]:
shuffCarData

,index,fuel_efficiency,maintenance,doors,type,size,safety,RATING
0,1571,crap,med,four,sports,med,high,JUNK
1,434,good,vhigh,two,sports,small,high,JUNK
2,791,good,low,three,sports,big,high,JUNK
3,8,excellent,vhigh,two,sports,big,high,JUNK
4,1517,crap,med,two,sports,med,high,JUNK
...,...,...,...,...,...,...,...,...
1723,273,excellent,med,four,sports,med,low,JUNK
1724,598,good,high,four,sports,med,med,JUNK
1725,354,excellent,low,three,sports,med,low,JUNK
1726,85,excellent,vhigh,many,sports,med,med,JUNK


In [7]:
shuffCarData.pop('index')

0       1571
1        434
2        791
3          8
4       1517
        ... 
1723     273
1724     598
1725     354
1726      85
1727    1112
Name: index, Length: 1728, dtype: int64

In [8]:
shuffCarData

,fuel_efficiency,maintenance,doors,type,size,safety,RATING
0,crap,med,four,sports,med,high,JUNK
1,good,vhigh,two,sports,small,high,JUNK
2,good,low,three,sports,big,high,JUNK
3,excellent,vhigh,two,sports,big,high,JUNK
4,crap,med,two,sports,med,high,JUNK
...,...,...,...,...,...,...,...
1723,excellent,med,four,sports,med,low,JUNK
1724,good,high,four,sports,med,med,JUNK
1725,excellent,low,three,sports,med,low,JUNK
1726,excellent,vhigh,many,sports,med,med,JUNK


In [9]:
# function definition: getting the counts (frequencies of occurrence)
def count(data,colname,label,target):    # for whichever data set, pick a column and one value in that column
                                         # and an output label (target) to compare against
    condition = (data[colname] == label) & (data['RATING'] == target)
    print(data[condition])
    return len(data[condition])

In [10]:
# list of predicted values
predicted = []

# dictionary to store probabilities
probabilities = {0:{},1:{},2:{},3:{}}

#length of current training set
train_len = int(splitRatio * len(shuffCarData))
print(train_len)

#splitting the data into training and test sets
training = shuffCarData.iloc[:train_len,:] # the first train_len rows, all columns

1468


In [11]:
# debugging step: looking at the training set
training

,fuel_efficiency,maintenance,doors,type,size,safety,RATING
0,crap,med,four,sports,med,high,JUNK
1,good,vhigh,two,sports,small,high,JUNK
2,good,low,three,sports,big,high,JUNK
3,excellent,vhigh,two,sports,big,high,JUNK
4,crap,med,two,sports,med,high,JUNK
...,...,...,...,...,...,...,...
1463,med,med,three,sedan,big,low,JUNK
1464,excellent,vhigh,two,sedan,big,high,JUNK
1465,med,vhigh,many,sports,big,med,JUNK
1466,med,high,three,wagon,med,med,OKAY


In [12]:
test_X = shuffCarData.iloc[train_len:,:-1] # features in the test set
test_y = shuffCarData.iloc[train_len:,-1]  # targets (the last entry in each row; i.e., the last column)

In [13]:
# debugging
print(test_X)

     fuel_efficiency maintenance  doors    type   size safety
1468            crap        high   four  sports  small   high
1469            good         low   four  sports  small   high
1470            crap         med   many   wagon    med    low
1471       excellent         med  three  sports    big    low
1472       excellent         low    two   sedan    med    med
...              ...         ...    ...     ...    ...    ...
1723       excellent         med   four  sports    med    low
1724            good        high   four  sports    med    med
1725       excellent         low  three  sports    med    low
1726       excellent       vhigh   many  sports    med    med
1727             med         med  three  sports    med   high

[260 rows x 6 columns]


In [14]:
# debugging
print(test_y)

1468    JUNK
1469    JUNK
1470    JUNK
1471    JUNK
1472    JUNK
        ... 
1723    JUNK
1724    JUNK
1725    JUNK
1726    JUNK
1727    JUNK
Name: RATING, Length: 260, dtype: object


In [15]:
# checking for dimension match
print(test_X.shape)
print(test_y.shape)

(260, 6)
(260,)


In [16]:
# counts of prior 'JUNK' and 'OKAY' labels (targets)
count_JUNK = count(training, 'RATING', 'JUNK', 'JUNK')
count_OKAY = count(training, 'RATING', 'OKAY', 'OKAY')
count_DANK = count(training, 'RATING', 'DANK', 'DANK')
count_good = count(training, 'RATING', 'good', 'good')

prior_JUNK = count_JUNK / len(training)
prior_OKAY = count_OKAY / len(training)
prior_DANK = count_DANK / len(training)
prior_good = count_good / len(training)

print('The prior \'JUNK\' count is', count_JUNK)
print('and \'OKAY\',',count_OKAY, '.')
print('and \'DANK\',',count_DANK, '.')
print('and \'good\',',count_good, '.')
print('The prior probability of \'JUNK\' is', prior_JUNK)
print('and of \'OKAY\',', prior_OKAY, '.')
print('and of \'DANK\',', prior_DANK, '.')
print('and of \'good\',', prior_good, '.')


     fuel_efficiency maintenance  doors    type   size safety RATING
0               crap         med   four  sports    med   high   JUNK
1               good       vhigh    two  sports  small   high   JUNK
2               good         low  three  sports    big   high   JUNK
3          excellent       vhigh    two  sports    big   high   JUNK
4               crap         med    two  sports    med   high   JUNK
...              ...         ...    ...     ...    ...    ...    ...
1461            good         low   many  sports    med    low   JUNK
1462            good        high   many  sports  small   high   JUNK
1463             med         med  three   sedan    big    low   JUNK
1464       excellent       vhigh    two   sedan    big   high   JUNK
1465             med       vhigh   many  sports    big    med   JUNK

[1018 rows x 7 columns]
     fuel_efficiency maintenance  doors   type   size safety RATING
6                med       vhigh    two  sedan    big    med   OKAY
11         

In [17]:
# TRAINING: learning the priors and class-conditionals from the training set
for col in training.columns[:-1]:
    probabilities[0][col] = {}
    probabilities[1][col] = {}
    probabilities[2][col] = {}
    probabilities[3][col] = {}

    if col == "fuel_efficiency":
        for k in fuel_efficienty_labels:
            count_k_JUNK = count(training, col, k, 'JUNK')
            count_k_OKAY = count(training, col, k, 'OKAY')
            count_k_DANK = count(training, col, k, 'DANK')
            count_k_good = count(training, col, k, 'good')
            probabilities[0][col][k] = count_k_JUNK / count_JUNK
            probabilities[1][col][k] = count_k_OKAY / count_OKAY
            probabilities[2][col][k] = count_k_DANK / count_DANK
            probabilities[3][col][k] = count_k_good / count_good
            probabilities[0][col][k] += 0.001 # throwing in a quick-and-dirty approximation of the
            probabilities[1][col][k] += 0.001 # "m estimate" for avoiding zeros in the product
            probabilities[2][col][k] += 0.001 
            probabilities[3][col][k] += 0.001 
    else:
        if col == "maintenance":
            for k in maintenance_labels:
                count_k_JUNK = count(training, col, k, 'JUNK')
                count_k_OKAY = count(training, col, k, 'OKAY')
                count_k_DANK = count(training, col, k, 'DANK')
                count_k_good = count(training, col, k, 'good')
                probabilities[0][col][k] = count_k_JUNK / count_JUNK
                probabilities[1][col][k] = count_k_OKAY / count_OKAY
                probabilities[2][col][k] = count_k_DANK / count_DANK
                probabilities[3][col][k] = count_k_good / count_good
                probabilities[0][col][k] += 0.001 # throwing in a quick-and-dirty approximation of the
                probabilities[1][col][k] += 0.001 # "m estimate" for avoiding zeros in the product
                probabilities[2][col][k] += 0.001 
                probabilities[3][col][k] += 0.001 
        else:
            if col == "type":
                for k in type_labels:
                    count_k_JUNK = count(training, col, k, 'JUNK')
                    count_k_OKAY = count(training, col, k, 'OKAY')
                    count_k_DANK = count(training, col, k, 'DANK')
                    count_k_good = count(training, col, k, 'good')
                    probabilities[0][col][k] = count_k_JUNK / count_JUNK
                    probabilities[1][col][k] = count_k_OKAY / count_OKAY
                    probabilities[2][col][k] = count_k_DANK / count_DANK
                    probabilities[3][col][k] = count_k_good / count_good
                    probabilities[0][col][k] += 0.001 # throwing in a quick-and-dirty approximation of the
                    probabilities[1][col][k] += 0.001 # "m estimate" for avoiding zeros in the product
                    probabilities[2][col][k] += 0.001 
                    probabilities[3][col][k] += 0.001 
            else:
                if col == "doors":
                    for k in doors_labels:
                        count_k_JUNK = count(training, col, k, 'JUNK')
                        count_k_OKAY = count(training, col, k, 'OKAY')
                        count_k_DANK = count(training, col, k, 'DANK')
                        count_k_good = count(training, col, k, 'good')
                        probabilities[0][col][k] = count_k_JUNK / count_JUNK
                        probabilities[1][col][k] = count_k_OKAY / count_OKAY
                        probabilities[2][col][k] = count_k_DANK / count_DANK
                        probabilities[3][col][k] = count_k_good / count_good
                        probabilities[0][col][k] += 0.001 # throwing in a quick-and-dirty approximation of the
                        probabilities[1][col][k] += 0.001 # "m estimate" for avoiding zeros in the product
                        probabilities[2][col][k] += 0.001 
                        probabilities[3][col][k] += 0.001 
                else:
                    if col == "size":
                        for k in size_labels:
                            count_k_JUNK = count(training, col, k, 'JUNK')
                            count_k_OKAY = count(training, col, k, 'OKAY')
                            count_k_DANK = count(training, col, k, 'DANK')
                            count_k_good = count(training, col, k, 'good')
                            probabilities[0][col][k] = count_k_JUNK / count_JUNK
                            probabilities[1][col][k] = count_k_OKAY / count_OKAY
                            probabilities[2][col][k] = count_k_DANK / count_DANK
                            probabilities[3][col][k] = count_k_good / count_good
                            probabilities[0][col][k] += 0.001 # throwing in a quick-and-dirty approximation of the
                            probabilities[1][col][k] += 0.001 # "m estimate" for avoiding zeros in the product
                            probabilities[2][col][k] += 0.001 
                            probabilities[3][col][k] += 0.001 
                    else:
                        for k in safety_labels:
                                count_k_JUNK = count(training, col, k, 'JUNK')
                                count_k_OKAY = count(training, col, k, 'OKAY')
                                count_k_DANK = count(training, col, k, 'DANK')
                                count_k_good = count(training, col, k, 'good')
                                probabilities[0][col][k] = count_k_JUNK / count_JUNK
                                probabilities[1][col][k] = count_k_OKAY / count_OKAY
                                probabilities[2][col][k] = count_k_DANK / count_DANK
                                probabilities[3][col][k] = count_k_good / count_good
                                probabilities[0][col][k] += 0.001 # throwing in a quick-and-dirty approximation of the
                                probabilities[1][col][k] += 0.001 # "m estimate" for avoiding zeros in the product
                                probabilities[2][col][k] += 0.001 
                                probabilities[3][col][k] += 0.001 

     fuel_efficiency maintenance  doors    type   size safety RATING
3          excellent       vhigh    two  sports    big   high   JUNK
7          excellent        high  three   sedan    med   high   JUNK
9          excellent        high  three   wagon    big    med   JUNK
10         excellent       vhigh   many   sedan    med    med   JUNK
16         excellent         med    two  sports    big   high   JUNK
...              ...         ...    ...     ...    ...    ...    ...
1449       excellent        high  three  sports  small   high   JUNK
1450       excellent        high    two   wagon    big   high   JUNK
1456       excellent         low   four   sedan  small    med   JUNK
1460       excellent         low   four  sports  small    med   JUNK
1464       excellent       vhigh    two   sedan    big   high   JUNK

[301 rows x 7 columns]
     fuel_efficiency maintenance  doors   type   size safety RATING
11         excellent         low   four  wagon    big   high   OKAY
20         e

In [18]:
import pprint

# Sample dictionary for demonstration
pp = pprint.PrettyPrinter(indent=4)

# Format each value in the nested dictionaries and print
formatted_probabilities = {
    key: {inner_key: {sub_key: "{:.3f}".format(sub_value) for sub_key, sub_value in inner_value.items()}
           for inner_key, inner_value in value.items()}
    for key, value in probabilities.items()
}

pp.pprint(formatted_probabilities)

{   0: {   'doors': {   'four': '0.238',
                        'many': '0.247',
                        'three': '0.248',
                        'two': '0.272'},
           'fuel_efficiency': {   'crap': '0.211',
                                  'excellent': '0.297',
                                  'good': '0.261',
                                  'med': '0.235'},
           'maintenance': {   'high': '0.269',
                              'low': '0.214',
                              'med': '0.220',
                              'vhigh': '0.301'},
           'safety': {'high': '0.235', 'low': '0.473', 'med': '0.295'},
           'size': {'big': '0.311', 'med': '0.313', 'small': '0.378'},
           'type': {'sedan': '0.252', 'sports': '0.473', 'wagon': '0.277'}},
    1: {   'doors': {   'four': '0.264',
                        'many': '0.273',
                        'three': '0.261',
                        'two': '0.206'},
           'fuel_efficiency': {   'crap': '0.231',
  

In [33]:
# TESTING: reading rows from the test set, checking the value of each feature and comparing with the target outcome
for row in range(len(test_X)):
    prod_JUNK = prior_JUNK
    prod_OKAY = prior_OKAY
    prod_DANK = prior_DANK
    prod_good = prior_good
        
    for feature in test_X.columns:
        # Check if the feature value exists in the probabilities dictionary:
        if test_X[feature].iloc[row] in probabilities[0][feature]:
            prod_JUNK *= probabilities[0][feature][test_X[feature].iloc[row]]
        else:
            prod_JUNK *= 0  # or some default value if the feature value is not found

        if test_X[feature].iloc[row] in probabilities[1][feature]:
            prod_OKAY *= probabilities[1][feature][test_X[feature].iloc[row]]
        else:
            prod_OKAY *= 0  # or some default value if the feature value is not found
            
        if test_X[feature].iloc[row] in probabilities[2][feature]:
            prod_DANK *= probabilities[2][feature][test_X[feature].iloc[row]]
        else:
            prod_DANK *= 0  # or some default value if the feature value is not found
            
        if test_X[feature].iloc[row] in probabilities[3][feature]:
            prod_good *= probabilities[3][feature][test_X[feature].iloc[row]]
        else:
            prod_good *= 0  # or some default value if the feature value is not found
        
    # predicting the outcome
    winner = 0;
    
    #if prod_JUNK > prod_OKAY & prod_JUNK > prod_DANK & prod_JUNK > prod_good:
    #    predicted.append('JUNK')
    #elif prod_JUNK < prod_OKAY & prod_OKAY > prod_DANK & prod_OKAY > prod_good:
    #    predicted.append('OKAY')
    #elif prod_JUNK < prod_DANK & prod_DANK > prod_OKAY & prod_DANK > prod_good:
    #    predicted.append('DANK')
    #else:
    #    predicted.append('good')
        
    answers = [prod_JUNK, prod_OKAY, prod_good, prod_DANK]
    winner= answers.index(max(answers))
    if(winner == 0):
        predicted.append('JUNK')
    elif(winner == 1):
        predicted.append('OKAY')
    elif(winner == 2):
        predicted.append('good')
    elif(winner == 3):
        predicted.append('DANK')
    
    
    
  
    
    print(predicted)
    
# Initialize correct and incorrect counts outside the loop
correct = 0
incorrect = 0

for j in range(len(test_y)):
    if predicted[j] == 'JUNK':
        if test_y.iloc[j] == 'JUNK':
            correct += 1
        else:
            incorrect += 1
    else:
        if test_y.iloc[j] == 'JUNK':
            incorrect += 1  # This should be incorrect since predicted is 'JUNK'
        else:
            correct += 1  # This should be correct since predicted is 'JUNK'
            
    if predicted[j] == 'OKAY':
        if test_y.iloc[j] == 'OKAY':
            correct += 1
        else:
            incorrect += 1
    else:
        if test_y.iloc[j] == 'OKAY':
            incorrect += 1  # This should be incorrect since predicted is 'JUNK'
        else:
            correct += 1  # This should be correct since predicted is 'JUNK'
            
    if predicted[j] == 'good':
        if test_y.iloc[j] == 'good':
            correct += 1
        else:
            incorrect += 1
    else:
        if test_y.iloc[j] == 'good':
            incorrect += 1  # This should be incorrect since predicted is 'JUNK'
        else:
            correct += 1  # This should be correct since predicted is 'JUNK'
            
    if predicted[j] == 'DANK':
        if test_y.iloc[j] == 'DANK':
            correct += 1
        else:
            incorrect += 1
    else:
        if test_y.iloc[j] == 'DANK':
            incorrect += 1  # This should be incorrect since predicted is 'JUNK'
        else:
            correct += 1  # This should be correct since predicted is 'JUNK'

print('The accuracy is', correct / (correct + incorrect))

['JUNK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'OKAY', 'JUNK', 'JUNK', 'JUNK', 'OKAY', 'JUNK', 'OKAY', 'JUNK', 'JUNK', 'OKAY', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'OKAY', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'OKAY', 'OKAY', 'JUNK', 'JUNK', 'OKAY', 'JUNK', 'JUNK', 'JUNK', 'OKAY', 'OKAY', 'OKAY', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'OKAY', 'OKAY', 'JUNK', 'OKAY', 'JUNK', 'JUNK', 'JUNK', 'OKAY', 'JUNK', 'OKAY', 'JUNK', 'JUNK', 'JUNK', 'OKAY', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'OKAY', 'OKAY', 'OKAY', 'OKAY', 'OKAY', 'good', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'OKAY', 'DANK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'OKAY', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'OKAY', 'JUNK', 'JUNK', 'JUNK', 'good', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'JUNK', 'OKAY', 'JUNK', 'JUNK', 'JUNK', 'OKAY', 'JUNK',